In [1]:
from line_solver import *
import numpy as np
GlobalConstants.set_verbose(VerboseLevel.STD)

In [2]:
# Fork-Join with Class Switching
model = Network('model')

In [3]:
# Create network
delay = Delay(model, 'Delay1')
delay2 = Delay(model, 'Delay2')
queue1 = Queue(model,'Queue1',SchedStrategy.PS)
queue2 = Queue(model,'Queue2',SchedStrategy.PS)
fork = Fork(model,'Fork')
join = Join(model,'Join', fork)

jobclass1 = ClosedClass(model, 'class1', 10, delay, 0)
jobclass2 = ClosedClass(model, 'class2', 10, delay, 0)

In [4]:
# Service configurations
queue1.set_service(jobclass1, Exp(1.0))
queue2.set_service(jobclass1, Exp(1.0))
delay.set_service(jobclass1, Exp(0.5))
delay.set_service(jobclass2, Exp(0.5))
delay2.set_service(jobclass2, Exp(2.0))

In [5]:
# Set routing matrix with class switching
P = model.init_routing_matrix()

# Class switching routing: jobclass1 -> jobclass2 -> jobclass1
P.set(jobclass1, jobclass2, delay, delay2, 1.0)    # Class switch: class1 -> class2 at delay
P.set(jobclass2, jobclass1, delay2, fork, 1.0)     # Class switch: class2 -> class1 at delay2

# Fork-join routing for class1
P.set(jobclass1, jobclass1, fork, queue1, 1.0)
P.set(jobclass1, jobclass1, fork, queue2, 1.0)
P.set(jobclass1, jobclass1, queue1, join, 1.0)
P.set(jobclass1, jobclass1, queue2, join, 1.0)
P.set(jobclass1, jobclass1, join, delay, 1.0)

model.link(P)

In [6]:
# Solve with multiple methods
solvers = [JMT(model, seed=23000), MVA(model)]
for i, solver in enumerate(solvers):
    print(f'SOLVER {i+1}: {solver.get_name()}')
    avgTable = solver.avg_table()

SOLVER 1: JMT


Station JobClass    QLen   Util   RespT  ResidT   ArvR   Tput
 Delay1   class1  1.9639 1.9639  1.9961  1.9961 0.9767 0.9767
 Delay1   class2  0.0000 0.0000  0.0000  0.0000 1.4687 1.4687
 Delay2   class1  0.0000 0.0000  0.0000  0.0000 0.0000 0.0000
 Delay2   class2  0.4939 0.4939  0.5027  0.5027 0.9752 0.9752
 Queue1   class1 12.9303 0.9914 12.0608 12.0608 0.9801 0.9801
 Queue1   class2  0.0000 0.0000  0.0000  0.0000 0.0000 0.0000
 Queue2   class1  9.2124 0.9612 10.2545 10.2545 0.9778 0.9778
 Queue2   class2  0.0000 0.0000  0.0000  0.0000 0.0000 0.0000
   Join   class1 12.9130 0.9999  6.5244  6.5244 0.9756 0.9756
   Join   class2  0.0000 0.0000  0.0000  0.0000 0.0000 0.0000
SOLVER 2: MVA
Station JobClass   QLen   Util  RespT  ResidT   ArvR   Tput
 Delay1   class1 1.6941 1.6941 2.0000  2.0000 0.8470 0.8470
 Delay1   class2 0.4322 0.4322 0.5000  0.5000 0.8645 0.8645
 Delay2   class1 1.6941 1.6941 2.0000  2.0000 0.8470 0.8470
 Delay2   class2 0.4322 0.4322 0.5000  0.5000 0.8645 0.8645
 Que